In [1]:
from datetime import datetime as dt
from datetime import timedelta
import logging
import importlib
import urllib
import unicodecsv as csv
from lxml import html
import requests
import pandas as pd
import matplotlib.pyplot as plt, numpy as np
import json
import sys
from requests.auth import HTTPProxyAuth
import time
import os
%matplotlib inline

In [2]:
#Script is written right now for before cleaned, single region. Must adapt for cleaned and multi-region.
#This will include changing "price" to "rent"
#QUESTION: Unclear what is actually being counted in 'score' variable

#cleandf = pd.read_csv('C:\\Users\\james\\Documents\\Berkeley_Docs\\Spring_17_Courses\\CP290 Data Lab\\github\\scraper2\\shared_room_scraper\\sfbay_merged_2017_04_10.csv', header=0) #just SF dataframe
cleandf = pd.read_csv('C:\\Users\\james\\Documents\\Berkeley_Docs\\Spring_17_Courses\\CP290 Data Lab\\github\\scraper2\\shared_room_scraper\\all_data_merged_2017_04_06.csv',error_bad_lines=False, header=0) #all listings dataframe
#determine rent per square foot


b'Skipping line 222647: expected 21 fields, saw 24\n'


In [3]:
cleandf.shape

(330869, 21)

In [4]:
cleandf.loc[cleandf.lat == 99, 'lat'] = 0
cleandf.loc[cleandf.lng == 99, 'lng'] = 0
cleandf.loc[cleandf.sqft == 0, 'sqft'] = np.nan

In [5]:
#For any duplicate post, we want to keep the version with the most information. Therefore, assign a score to each post and 
#give one point for square footage, lat, long and price
#OLD FORMULA
#cleandf['score']=cleandf.astype(bool).sum(axis=1)

cleandf['price_exists'] = cleandf['price']>0
cleandf['sqft_exists'] = cleandf['sqft']>0
cleandf['lat_exists'] = cleandf['lat']>0
cleandf['lng_exists'] = cleandf['lng']<0
cleandf['score'] = cleandf[['price_exists','sqft_exists','lat_exists','lng_exists']].astype(bool).sum(axis=1)

In [6]:
#Sort rows by score
cleandf = cleandf.sort_values(by='score',ascending=False)

In [7]:
cleandf.shape

(330869, 26)

In [8]:
cleandf.tail()

,pid,dt,url,title,price,neighb,sqft,lat,lng,accuracy,...,private_room,bath_known,private_bath,parking_known,onsite_parking,price_exists,sqft_exists,lat_exists,lng_exists,score
172537,6067818749,2017-04-03 08:04,http://newyork.craigslist.org/brx/roo/60678187...,NESECITA DINERO PARA PAGAR TU RENTA ?,NaN,RENTA TU CUARTO RÁPIDO CON NOSOTROS,NaN,0.0,0.0,99,...,True,True,False,False,False,False,False,False,False,0
172538,6061818165,2017-04-03 08:02,http://newyork.craigslist.org/brx/roo/60618181...,TIENES CUARTOS VACIOS ?,NaN,LLAMENOS,NaN,0.0,0.0,99,...,True,True,False,True,False,False,False,False,False,0
172539,6061812984,2017-04-03 08:02,http://newyork.craigslist.org/brx/roo/60618129...,BUSCAMOS CUARTOS PARA RENTAR,NaN,EN TODO EL BRONX Y MANHATTAN,NaN,0.0,0.0,99,...,True,True,False,True,False,False,False,False,False,0
172544,6040435458,2017-04-03 08:01,http://newyork.craigslist.org/brx/roo/60404354...,ROOMS WANTED,NaN,BRONX-MANHATTAN-QUEENS-BROOKLYN,NaN,0.0,0.0,99,...,True,True,False,False,False,False,False,False,False,0
0,6065227510,2017-04-01 10:08,http://abilene.craigslist.org/roo/6065227510.html,Female Roommate Wanted,NaN,Odessa,NaN,0.0,0.0,99,...,False,True,False,False,False,False,False,False,False,0


In [9]:
#List as 'true' if the PID, body_text, or the title are duplicates
cleandf['body_text_duplicate']=cleandf.duplicated('body_text')
cleandf['title_duplicate']=cleandf.duplicated('title')
cleandf['PID_duplicate']=cleandf.duplicated('pid')


In [10]:
cleandf_unique = cleandf.drop_duplicates('pid')
cleandf_unique.shape

(99454, 29)

In [11]:
#Keep rows with highest score. Drop rows with duplicate PIDs, Duplicate Titles, Duplicate Posting Bodies
cleandf_unique = cleandf_unique.drop_duplicates('body_text')
cleandf_unique.shape

(75977, 29)

In [12]:
cleandf_unique = cleandf_unique.drop_duplicates('title')
cleandf_unique.shape

(62032, 29)

## Creating new fields for analysis

In [13]:
cleandf_unique['dt'] = pd.to_datetime(cleandf_unique['dt'], format='%Y-%m-%d')
cleandf_unique['region'] = cleandf_unique['url'].str.extract('http://(.*).craigslist.org', expand=False)
cleandf_unique['day_of_week'] = cleandf_unique['dt'].apply(lambda x: x.weekday())
cleandf_unique['rent_sqft'] = cleandf_unique['price'] / cleandf['sqft']

In [14]:
#counting listings per day (right now inlcudes time, need to remove)
listings_per_date = cleandf_unique['dt'].value_counts()

## Descriptive Stats and More Filtering

In [15]:
cleandf_unique.describe()

,pid,price,sqft,lat,lng,accuracy,score,day_of_week,rent_sqft
count,6.203200e+04,5.257500e+04,1.619500e+04,62032.000000,62032.000000,62032.000000,62032.000000,62032.000000,15728.000000
mean,6.065521e+09,8.040899e+03,1.585170e+03,29.699271,-75.462708,32.142023,2.687452,2.716711,20.757369
std,1.153673e+07,2.606233e+05,6.594811e+04,15.932357,42.825087,35.142094,1.091148,2.182855,969.456528
min,6.025119e+09,1.000000e+00,1.000000e+00,0.000000,-158.065796,0.000000,0.000000,0.000000,0.000107
25%,6.061505e+09,4.950000e+02,1.850000e+02,28.467100,-117.089474,10.000000,2.000000,1.000000,0.434334
50%,6.070064e+09,6.500000e+02,9.000000e+02,36.797800,-80.292600,22.000000,3.000000,2.000000,1.045455
75%,6.073341e+09,9.000000e+02,1.410000e+03,40.674242,-72.673037,22.000000,3.000000,5.000000,3.750000
max,6.076942e+09,2.250150e+07,8.388607e+06,64.878089,0.000000,99.000000,4.000000,6.000000,85723.928571


In [16]:
unique_wprice = cleandf_unique[cleandf_unique['price'] > 0]

In [17]:
# in this cell, define the values by which we will filter the 3 columns. This will vary depending on the sample we're looking at. 
upper_percentile = 0.997
lower_percentile = 0.01

# how many rows would be within the upper and lower percentiles?
upper = int(len(unique_wprice) * upper_percentile)
lower = int(len(unique_wprice) * lower_percentile)

# get the rent/sqft values at the upper and lower percentiles
rent_sqft_sorted = unique_wprice['rent_sqft'].sort_values(ascending=True, inplace=False)
upper_rent_sqft = rent_sqft_sorted.iloc[upper]
lower_rent_sqft = rent_sqft_sorted.iloc[lower]

# get the rent values at the upper and lower percentiles
rent_sorted = unique_wprice['price'].sort_values(ascending=True, inplace=False)
upper_rent = rent_sorted.iloc[upper]
lower_rent = rent_sorted.iloc[lower]

# get the sqft values at the upper and lower percentiles
sqft_sorted = unique_wprice['sqft'].sort_values(ascending=True, inplace=False)
upper_sqft = sqft_sorted.iloc[upper]
lower_sqft = sqft_sorted.iloc[lower]

print('valid rent_sqft range:', [lower_rent_sqft, upper_rent_sqft])
print('valid rent range:', [lower_rent, upper_rent])
print('valid sqft range:', [lower_sqft, upper_sqft])

valid rent_sqft range: [0.14166666666666666, nan]
valid rent range: [50.0, 5500.0]
valid sqft range: [100.0, nan]


In [19]:
#Filtering out rows with unreasonable rent prices 
rent_mask = (unique_wprice['price'] > lower_rent) & (unique_wprice['price'] < upper_rent)

In [20]:
filtered_listings = pd.DataFrame(unique_wprice[rent_mask])

## Filtering out listings with "0" latitudes

In [21]:
lat_mask = (filtered_listings['lat'] > 0)

In [22]:
filtered_listings = pd.DataFrame(filtered_listings[lat_mask])

In [23]:
filtered_listings.describe()

,pid,price,sqft,lat,lng,accuracy,score,day_of_week,rent_sqft
count,4.175000e+04,41750.000000,1.363300e+04,41750.000000,41750.000000,41750.000000,41750.000000,41750.000000,13633.000000
mean,6.065781e+09,775.042563,1.640565e+03,37.740904,-95.694069,14.013581,3.326539,2.692910,3.574683
std,1.138951e+07,404.784149,7.185588e+04,4.785129,20.146424,7.242707,0.468953,2.180999,20.646918
min,6.025125e+09,51.000000,1.000000e+00,4.816900,-158.065796,0.000000,3.000000,0.000000,0.000107
25%,6.062278e+09,500.000000,1.800000e+02,34.061400,-118.177104,7.000000,3.000000,1.000000,0.458333
50%,6.070171e+09,700.000000,9.000000e+02,38.686500,-87.911085,15.000000,3.000000,2.000000,1.087500
75%,6.073427e+09,950.000000,1.400000e+03,40.743600,-76.558689,22.000000,4.000000,5.000000,3.846154
max,6.076942e+09,5100.000000,8.388607e+06,64.878089,-4.817498,40.000000,4.000000,6.000000,1000.000000


## Viewing Results by Region

In [ ]:
#grouped = cleandf_unique.groupby('region')

In [24]:
countdata = filtered_listings['region'].value_counts().head(30)

In [25]:
countdata

newyork         5678
sfbay           4751
losangeles      2712
washingtondc    2474
boston          1501
sandiego        1428
orangecounty     977
miami            952
chicago          901
seattle          873
portland         720
denver           692
atlanta          594
philadelphia     554
phoenix          532
houston          503
inlandempire     490
austin           482
baltimore        434
orlando          434
sacramento       429
newjersey        424
lasvegas         389
dallas           367
tampa            352
minneapolis      283
raleigh          277
norfolk          263
boulder          234
santabarbara     223
Name: region, dtype: int64

In [37]:
filtered_listings[filtered_listings['region']=='boston'].describe()

,pid,price,sqft,lat,lng,accuracy,score,day_of_week,rent_sqft
count,1.501000e+03,1501.000000,421.000000,1501.000000,1501.000000,1501.000000,1501.000000,1501.000000,421.000000
mean,6.066421e+09,901.688874,927.859857,42.367702,-71.127071,13.588274,3.280480,3.101932,8.923746
std,1.032795e+07,329.980912,1045.223201,0.141190,0.178810,6.659007,0.449383,2.284353,70.451256
min,6.026378e+09,125.000000,1.000000,39.947800,-74.903600,0.000000,3.000000,0.000000,0.107143
25%,6.063413e+09,700.000000,150.000000,42.331500,-71.147706,7.000000,3.000000,1.000000,0.720000
50%,6.070014e+09,850.000000,860.000000,42.355442,-71.109653,15.000000,3.000000,4.000000,1.384909
75%,6.073095e+09,1050.000000,1300.000000,42.394300,-71.075084,20.000000,4.000000,5.000000,5.000000
max,6.076534e+09,3200.000000,7500.000000,42.850700,-70.642000,23.000000,4.000000,6.000000,1000.000000


## Visualization

In [ ]:
x = filtered_listings['price']

In [ ]:
x = x.astype('int')

In [ ]:
plt.figure(1, figsize=(10,6) )
plt.suptitle('Rent Price Distribution')
plt.xlabel('Price')
plt.ylabel('Number of Listings')
ax = plt.hist(x, bins=50, alpha=.4, color='cyan')
plt.show()

## Counting Words in body text

In [ ]:
np.sum(filtered_listings['body_text'].str.contains('vegetarian'))

In [ ]:
#filtered_listings.to_csv('sfbay_filtered4_10.csv')

In [ ]:
# calculate the ratios of unique to duplicate listings for each region
listings_ratios = pd.DataFrame()

# number of total listings for each region
listings_ratios['cleandf'] = cleandf['region'].value_counts()

# number of duplicate listings for each region (ie, listings that share a pid with at least one other listing)
listings_ratios['duplicate_listings'] = duplicate_listings['region'].value_counts()

# number of unique listings for the region (ie, none share a pid with another listing)
listings_ratios['unique_listings'] = unique_listings['region'].value_counts()

# percent of this region's listings that are duplicates
listings_ratios['duplicate_ratio'] = listings_ratios['duplicate_listings'] / listings_ratios['all_listings']

# percent of this region's listings that are unique (ie, not duplicates)
listings_ratios['unique_ratio'] = listings_ratios['unique_listings'] / listings_ratios['all_listings']

listings_ratios.head()